In [3]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

In [4]:
artista = 'embrulhaPraViagem'

In [5]:
colunas_data = {
    'Impressões':'Impressões',
    'Visualizações':'Visualizações',
    'Taxa de cliques de impressões (%)':'CTR (%)',
    'Duração média da visualização':'Tempo Médio Assistido',
    'Porcentagem visualizada média (%)':'Porcentagem Média Assistida',
    'RPM (USD)':'RPM (USD)',
    'Marcações "Não gostei"':'Porcentagem de "Não Gostei"',
    'Inscritos':'Saldo de Inscritos'
}

colunas_conteudo = {
    'Vídeos publicados':'Número de conteúdos Novo',
    'Impressões':'Impressões Novo',
    'Visualizações':'Visualizações Novo',
    'Taxa de cliques de impressões (%)':'CTR (%) Novo',
    'Duração média da visualização':'Tempo Médio Assistido Novo',
    'Porcentagem visualizada média (%)':'Porcentagem Média Assistida Novo',
    'RPM (USD)':'RPM (USD) Novo',
    'Marcações "Não gostei"':'Porcentagem de "Não Gostei" Novo',
    'Inscritos':'Saldo de Inscritos Novo',
    'Minutagem média':'Tamanho médio do vídeo (min) Novo'
    
}

colunas_receita = {
    'Receita estimada (USD)_x':'Receita Total',
    'Receita estimada (USD)_y':'Receita Conteúdo novo (Soma)',
    'Conteúdo Velho':'Receita Conteúdo velho (Soma)'

}

colunas_shorts = {
    'Vídeos publicados':'Número de Shorts',
    'Impressões':'Impressões Shorts',
    'Visualizações':'Visualizações Shorts',
    'Porcentagem visualizada média (%)':'Retenção (%) Shorts',
    'RPM (USD)':'RPM Shorts',
    'Duração média da visualização':'Tempo Médio Assistido Shorts',
    'Minutagem média':'Tamanho de Vídeo Shorts',
    'Inscritos':'Saldo de Inscritos Shorts'
}

colunas_comunidade = {
    'Post text':'Texto da postagem',
    'Texto da publicação':'Texto da postagem',
    'Post publish time':'Horário de publicação da postagem',
    'Hora da publicação':'Horário de publicação da postagem',
    'Post impressions':'Impressões da postagem',
    'Impressões da publicação':'Impressões da postagem',
    'Post likes':'"Gostei" da postagem',
    'Gostos da publicação':'"Gostei" da postagem',
    'Post like rate (%)':'Média de "Gostei" da postagem (%)',
    """Média de """"Gostei"""" da postagem (%)""":'Média de "Gostei" da postagem (%)',
    'Percentagem de Gostos da publicação (%)':'Média de "Gostei" da postagem (%)',

}

colunas_receita_ordenada = ['Receita Conteúdo novo (Soma)', 'Receita Conteúdo velho (Soma)', 'Receita Total'] 

In [6]:
def format_as_percentage(value):
    return '{:.2%}'.format(value)

In [7]:
f = open("dados_full/"+artista+"/sub.txt", "r")
subs = int(''.join((f.readline()).split('.')))
f.close()

# Tabela de Análise de Conteúdo (DATA/SUBS)

In [254]:
data_1 = pd.read_csv(f'dados_full/{artista}/subs.csv')
data_1 = data_1.drop(data_1.index[0])
data_1['Data'] = pd.to_datetime(data_1['Data'])
data_1['Data'] = data_1['Data'].dt.to_period('M')
data = data_1.copy()
data = data[['Data','Impressões', 'Visualizações','Taxa de cliques de impressões (%)', 'Duração média da visualização', 'Porcentagem visualizada média (%)', 'RPM (USD)', 'Marcações "Não gostei"', 'Inscritos']]
data['Marcações "Não gostei"'] = (data['Marcações "Não gostei"'] / data['Visualizações'])
data['Duração média da visualização'] = pd.to_timedelta(data['Duração média da visualização'])
data['Taxa de cliques de impressões (%)'] = data['Taxa de cliques de impressões (%)'] / 100
data['Porcentagem visualizada média (%)'] = data['Porcentagem visualizada média (%)'] / 100
data_mes = data.copy()
data_mes.sort_values(by='Data',inplace=True, ascending= False)
mes = data_mes['Data'].iloc[0]


data_impressoes = data['Impressões'].mean()
data_visualizacoes = data['Visualizações'].mean()
data_ctr = data['Taxa de cliques de impressões (%)'].mean()
data_tempo_assistido = data['Duração média da visualização'].mean()
data_tempo_assistido = data_tempo_assistido.seconds
data_tempo_assistido = '{:02d}:{:02d}'.format(data_tempo_assistido // 60, data_tempo_assistido % 60)
data_porcentagem_assistida = data['Porcentagem visualizada média (%)'].mean()
data_rpm = data['RPM (USD)'].mean()
data_nao_gostei = data['Marcações "Não gostei"'].mean()


data['Duração média da visualização'] = pd.to_timedelta(data['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(data['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
data.sort_values(by='Data', inplace=True, ascending=False)
data_cumulative = subs - data['Inscritos'].cumsum()
data['Inscritos'] = data_cumulative.shift(1).fillna(subs).astype(int)
data_inscritos = data['Inscritos'].mean()
data.sort_values(by='Data', inplace=True, ascending=True)
data.rename(columns=colunas_data, inplace=True)
data.set_index('Data', inplace = True)
data = data.T
data.insert(0, 'Média', [data_impressoes, data_visualizacoes, data_ctr, data_tempo_assistido, data_porcentagem_assistida, data_rpm, data_nao_gostei, data_inscritos])
data.reset_index(inplace= True)
data.rename(columns={'index':'Análise Conteúdos'}, inplace = True)
data.fillna(0, inplace=True)
data

Data,Análise Conteúdos,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Impressões,28104918.166667,37270352,33207569,26512256,18850627,24290205,28498500
1,Visualizações,4310879.333333,5336846,4759200,4742339,3752755,3175314,4098822
2,CTR (%),0.088583,0.0987,0.0931,0.0923,0.0836,0.0798,0.084
3,Tempo Médio Assistido,02:50,03:00,02:52,02:40,02:05,03:13,03:11
4,Porcentagem Média Assistida,0.5957,0.6318,0.6285,0.5343,0.5127,0.6226,0.6443
5,RPM (USD),1.178667,1.71,1.278,1.028,0.791,1.222,1.043
6,"Porcentagem de ""Não Gostei""",0.000301,0.000329,0.000322,0.000278,0.000231,0.000334,0.000313
7,Saldo de Inscritos,857183.833333,830699,844555,854256,861515,868978,883100


In [255]:
data_temp = data.copy()
data_temp.set_index('Análise Conteúdos', inplace = True)
data_temp.loc['Tempo Médio Assistido'] = data_temp.loc['Tempo Médio Assistido'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))

data_desvio = data_temp.copy()
data_desvio.iloc[:, 1:] = (data_desvio.iloc[:, 1:].div(data_desvio['Média'], axis=0) - 1)
data_desvio.drop('Média', axis=1, inplace=True)
data_desvio.reset_index(drop=True, inplace=True)
data_desvio.insert(0, 'Análise Conteúdos', data['Análise Conteúdos'])
data_desvio.fillna(0, inplace=True)
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].replace('.', ',', regex=True)
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].astype(float)

In [256]:
#Formatação de porcentagens conteúdo
data.set_index('Análise Conteúdos', inplace=True)
data.loc['CTR (%)'] = data.loc['CTR (%)'].map(format_as_percentage)
data.loc['Porcentagem de "Não Gostei"'] = data.loc['Porcentagem de "Não Gostei"'].map(format_as_percentage)
data.loc['Porcentagem Média Assistida'] = data.loc['Porcentagem Média Assistida'].map(format_as_percentage)
data.reset_index(inplace=True)
data = data.astype(str)

#Formatação de porcentagens desvio
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
data_desvio_formatado = data_desvio.iloc[:, 1:].map(format_as_percentage)
data_desvio_formatado = pd.concat([data_desvio['Análise Conteúdos'], data_desvio_formatado], axis=1)
data_desvio_formatado = data_desvio_formatado.astype(str)

In [257]:
data_desvio_formatado

,Análise Conteúdos,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Impressões,32.61%,18.16%,-5.67%,-32.93%,-13.57%,1.40%
1,Visualizações,23.80%,10.40%,10.01%,-12.95%,-26.34%,-4.92%
2,CTR (%),11.42%,5.10%,4.20%,-5.63%,-9.92%,-5.17%
3,Tempo Médio Assistido,5.88%,1.18%,-5.88%,-26.47%,13.53%,12.35%
4,Porcentagem Média Assistida,6.06%,5.51%,-10.31%,-13.93%,4.52%,8.16%
5,RPM (USD),45.08%,8.43%,-12.78%,-32.89%,3.68%,-11.51%
6,"Porcentagem de ""Não Gostei""",9.14%,6.97%,-7.62%,-23.24%,10.79%,3.96%
7,Saldo de Inscritos,-3.09%,-1.47%,-0.34%,0.51%,1.38%,3.02%


# Tabela de Novos Conteúdos

In [12]:
def processar_dados(artista, num_videos):
    dados = []
    for i in range(1, num_videos + 1):
        arquivo = f'dados_full/{artista}/videos{i}.csv'
        df = pd.read_csv(arquivo)
        df['Horário de publicação do vídeo'] = pd.to_datetime(df['Horário de publicação do vídeo'])
        df['Data'] = df['Horário de publicação do vídeo'].dt.to_period('M')
        df.sort_values(by='Data', ascending=False, inplace=True)

        ultimo_mes = df['Horário de publicação do vídeo'].max().to_period('M')
        meses_selecionados = pd.period_range(end=ultimo_mes, periods=4, freq='M')
        df = df[df['Horário de publicação do vídeo'].dt.to_period('M').isin(meses_selecionados)]

        df['Duração média da visualização'] = pd.to_timedelta(df['Duração média da visualização']).apply(lambda x: x.total_seconds())
        df.set_index('Data', inplace=True)
        df = df.dropna(subset=['Horário de publicação do vídeo'])
        dados.append(df)
    return pd.concat(dados)

num_videos = 6
conteudo_1 = processar_dados(artista, num_videos)
conteudo_1.reset_index(inplace=True)
conteudo_1['Data'].unique()

<PeriodArray>
['2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04']
Length: 6, dtype: period[M]

In [13]:
# Contagem de linhas por valor na coluna 'Data'
videos_publicados = conteudo_1['Data'].value_counts().reset_index()
videos_publicados.columns = ['Data', 'Vídeos Publicados']
# videos_publicados.loc[[4], 'Vídeos Publicados'] = 0
videos_publicados.sort_values(by='Data', inplace=True)
videos_publicados

,Data,Vídeos Publicados
0,2023-11,13
1,2023-12,13
2,2024-01,13
3,2024-02,13
4,2024-03,12
5,2024-04,12


In [14]:
conteudo_1.set_index('Data', inplace=True)
conteudo = conteudo_1.copy()
conteudo_1 = conteudo.groupby(pd.Grouper(freq='M')).agg({'Receita estimada (USD)' : 'sum'})
conteudo_1 = conteudo_1.reset_index()


conteudo = conteudo.groupby(pd.Grouper(freq='M')).agg({'Impressões':'sum', 'Visualizações':'sum', 'Taxa de cliques de impressões (%)':'mean', 'Duração média da visualização':'mean', 'Porcentagem visualizada média (%)':'mean', 'RPM (USD)':'mean', 'Marcações "Não gostei"':'mean', 'Inscritos':'sum'})
conteudo['Minutagem média'] = (conteudo['Duração média da visualização'] * 100) / conteudo['Porcentagem visualizada média (%)']
conteudo['Marcações "Não gostei"'] = (conteudo['Marcações "Não gostei"'] / conteudo['Visualizações']) * 100
conteudo['Taxa de cliques de impressões (%)'] = conteudo['Taxa de cliques de impressões (%)'] / 100
conteudo['Porcentagem visualizada média (%)'] = conteudo['Porcentagem visualizada média (%)'] / 100
conteudo_2 = conteudo.copy()

conteudo

,Impressões,Visualizações,Taxa de cliques de impressões (%),Duração média da visualização,Porcentagem visualizada média (%),RPM (USD),"Marcações ""Não gostei""",Inscritos,Minutagem média
Data,,,,,,,,,
2023-11,15967177,2681142,0.101785,154.615385,0.579115,2.186692,0.002596,8270,266.985455
2023-12,12550570,2244480,0.102777,166.000000,0.536569,1.377692,0.003057,5348,309.372939
2024-01,13880306,3178237,0.101023,147.615385,0.465900,1.018538,0.002181,4824,316.839203
2024-02,8604633,2513772,0.093431,130.384615,0.488485,0.909846,0.001790,3524,266.916524
2024-03,12522721,1652577,0.096808,165.666667,0.593208,1.320833,0.003883,3691,279.272319
2024-04,12752304,2098517,0.093033,149.583333,0.577875,0.991083,0.002700,6322,258.850674


In [260]:
conteudo_1.set_index('Data', inplace=True)
conteudo = conteudo_1.copy()
conteudo_1 = conteudo.groupby(pd.Grouper(freq='M')).agg({'Receita estimada (USD)' : 'sum'})
conteudo_1 = conteudo_1.reset_index()


conteudo = conteudo.groupby(pd.Grouper(freq='M')).agg({'Impressões':'sum', 'Visualizações':'sum', 'Taxa de cliques de impressões (%)':'mean', 'Duração média da visualização':'mean', 'Porcentagem visualizada média (%)':'mean', 'RPM (USD)':'mean', 'Marcações "Não gostei"':'mean', 'Inscritos':'sum'})
conteudo['Minutagem média'] = (conteudo['Duração média da visualização'] * 100) / conteudo['Porcentagem visualizada média (%)']
conteudo['Marcações "Não gostei"'] = (conteudo['Marcações "Não gostei"'] / conteudo['Visualizações']) * 100
conteudo['Taxa de cliques de impressões (%)'] = conteudo['Taxa de cliques de impressões (%)'] / 100
conteudo['Porcentagem visualizada média (%)'] = conteudo['Porcentagem visualizada média (%)'] / 100
conteudo_2 = conteudo.copy()

conteudo_impressoes = conteudo['Impressões'].mean()
conteudo_visualizacoes = conteudo['Visualizações'].mean()
conteudo_ctr = conteudo['Taxa de cliques de impressões (%)'].mean()
conteudo_tempo_assistido = conteudo['Duração média da visualização'].mean()
conteudo_tempo_assistido = '{:02d}:{:02d}'.format(int(conteudo_tempo_assistido // 60), int(conteudo_tempo_assistido % 60))
conteudo_porcentagem_assistida = conteudo['Porcentagem visualizada média (%)'].mean()
conteudo_rpm = conteudo['RPM (USD)'].mean()
conteudo_nao_gostei = conteudo['Marcações "Não gostei"'].mean()
conteudo_inscritos = conteudo['Inscritos'].mean()


conteudo['Duração média da visualização'].fillna(0, inplace=True)
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].astype(int)
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].apply(lambda x: int(x))
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
conteudo['Duração média da visualização'] = pd.to_timedelta(conteudo['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(conteudo['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
conteudo['Minutagem média'] = conteudo['Minutagem média'] / 60
conteudo_minutagem = conteudo['Minutagem média'].mean()
conteudo = conteudo.reset_index()
conteudo = pd.merge(conteudo, videos_publicados[['Data', 'Vídeos Publicados']], on = 'Data', how = 'left')
conteudo_pubs = conteudo['Vídeos Publicados'].mean()
cols = conteudo.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[1:-1]
conteudo = conteudo[cols]
conteudo.sort_values(by = 'Data', inplace = True)
conteudo.rename(columns=colunas_conteudo, inplace=True)
conteudo.set_index('Data', inplace = True)
conteudo = conteudo.T
conteudo.insert(0, 'Média', [conteudo_pubs,conteudo_impressoes, conteudo_visualizacoes, conteudo_ctr, conteudo_tempo_assistido, conteudo_porcentagem_assistida, conteudo_rpm, conteudo_nao_gostei, conteudo_inscritos, conteudo_minutagem])
conteudo.reset_index(inplace = True)
conteudo.rename(columns={'index':'Analise de Conteúdo Novo'}, inplace = True)
conteudo.fillna(0, inplace=True)
conteudo


Data,Analise de Conteúdo Novo,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Vídeos Publicados,12.666667,13,13,13,13,12,12
1,Impressões Novo,12712951.833333,15967177,12550570,13880306,8604633,12522721,12752304
2,Visualizações Novo,2394787.5,2681142,2244480,3178237,2513772,1652577,2098517
3,CTR (%) Novo,0.098143,0.101785,0.102777,0.101023,0.093431,0.096808,0.093033
4,Tempo Médio Assistido Novo,02:32,02:34,02:46,02:27,02:10,02:45,02:29
5,Porcentagem Média Assistida Novo,0.540192,0.579115,0.536569,0.4659,0.488485,0.593208,0.577875
6,RPM (USD) Novo,1.300781,2.186692,1.377692,1.018538,0.909846,1.320833,0.991083
7,"Porcentagem de ""Não Gostei"" Novo",0.002701,0.002596,0.003057,0.002181,0.00179,0.003883,0.0027
8,Saldo de Inscritos Novo,5329.833333,8270,5348,4824,3524,3691,6322
9,Tamanho médio do vídeo (min) Novo,4.717325,4.449758,5.156216,5.280653,4.448609,4.654539,4.314178


In [261]:
conteudo_temp = conteudo.copy()
conteudo_temp.set_index('Analise de Conteúdo Novo', inplace = True)
conteudo_temp.loc['Tempo Médio Assistido Novo'] = conteudo_temp.loc['Tempo Médio Assistido Novo'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))
conteudo_desvio = conteudo_temp.copy()
conteudo_desvio.iloc[:, 1:] = conteudo_desvio.iloc[:, 1:].div(conteudo_desvio['Média'], axis=0) - 1
conteudo_desvio.drop('Média', axis=1, inplace=True)
conteudo_desvio.reset_index(drop=True, inplace=True)
conteudo_desvio.insert(0, 'Análise de Conteúdo Novo', conteudo['Analise de Conteúdo Novo'])
conteudo_desvio.fillna(0, inplace=True)
conteudo_desvio

Data,Análise de Conteúdo Novo,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Vídeos Publicados,0.026316,0.026316,0.026316,0.026316,-0.052632,-0.052632
1,Impressões Novo,0.255977,-0.012773,0.091824,-0.32316,-0.014964,0.003095
2,Visualizações Novo,0.119574,-0.062764,0.327148,0.049685,-0.309927,-0.123715
3,CTR (%) Novo,0.037107,0.047218,0.029347,-0.048012,-0.013598,-0.052062
4,Tempo Médio Assistido Novo,0.013158,0.092105,-0.032895,-0.144737,0.085526,-0.019737
5,Porcentagem Média Assistida Novo,0.072055,-0.006707,-0.137529,-0.095721,0.098143,0.069758
6,RPM (USD) Novo,0.681061,0.059127,-0.216979,-0.300539,0.015416,-0.238086
7,"Porcentagem de ""Não Gostei"" Novo",-0.038787,0.131723,-0.19271,-0.337294,0.437415,-0.000347
8,Saldo de Inscritos Novo,0.551643,0.003408,-0.094906,-0.338816,-0.307483,0.186153
9,Tamanho médio do vídeo (min) Novo,-0.05672,0.093038,0.119417,-0.056964,-0.01331,-0.085461


In [262]:
#Formatação de porcentagens conteúdo
conteudo.set_index('Analise de Conteúdo Novo', inplace=True)
conteudo.loc['CTR (%) Novo'] = conteudo.loc['CTR (%) Novo'].map(format_as_percentage)
conteudo.loc['Porcentagem de "Não Gostei" Novo'] = conteudo.loc['Porcentagem de "Não Gostei" Novo'].map(format_as_percentage)
conteudo.loc['Porcentagem Média Assistida Novo'] = conteudo.loc['Porcentagem Média Assistida Novo'].map(format_as_percentage)
conteudo.reset_index(inplace=True)
conteudo = conteudo.astype(str)

#Formatação de porcentagens desvio
conteudo_desvio.iloc[:, 1:] = conteudo_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
conteudo_desvio_formatado = conteudo_desvio.iloc[:, 1:].map(format_as_percentage)
conteudo_desvio_formatado = pd.concat([conteudo_desvio['Análise de Conteúdo Novo'], conteudo_desvio_formatado], axis=1)
conteudo_desvio_formatado = conteudo_desvio_formatado.astype(str)

# Tabela de Receita

In [263]:
receita = data_1[['Data', 'Receita estimada (USD)']]
receita = pd.merge(receita, conteudo_1[['Data', 'Receita estimada (USD)']], on = 'Data', how = 'left')
receita['Conteúdo Velho'] = receita['Receita estimada (USD)_x'] - receita['Receita estimada (USD)_y']

receita_total = receita['Receita estimada (USD)_x'].mean()
receita_novo = receita['Receita estimada (USD)_y'].mean()
receita_velho = receita['Conteúdo Velho'].mean()


receita.sort_values(by = 'Data', inplace = True)
receita.rename(columns=colunas_receita, inplace=True)
receita.set_index('Data', inplace = True)
receita = receita[colunas_receita_ordenada]
receita.insert(3,'Receita % Novo',(receita['Receita Conteúdo novo (Soma)']/receita['Receita Total']))
receita.insert(4,'Receita % Velho',(receita['Receita Conteúdo velho (Soma)']/receita['Receita Total']))

receita_percent_novo = receita['Receita % Novo'].mean()
receita_percent_velho = receita['Receita % Velho'].mean()

receita = receita.T
receita.insert(0, 'Média', [receita_novo, receita_velho, receita_total, receita_percent_novo, receita_percent_velho])
receita.reset_index(inplace = True)
receita.rename(columns={'index':'Receita (USD)'}, inplace = True)
receita.fillna(0, inplace=True)
receita

Data,Receita (USD),Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Receita Conteúdo novo (Soma),3004.055833,5330.671000,3115.888000,3257.471000,1686.071000,2326.096000,2308.13800
1,Receita Conteúdo velho (Soma),2198.198333,3797.711000,2968.443000,1619.147000,1282.777000,1554.605000,1966.50700
2,Receita Total,5202.254167,9128.382000,6084.331000,4876.618000,2968.848000,3880.701000,4274.64500
3,Receita % Novo,0.578557,0.583967,0.512117,0.667977,0.567921,0.599401,0.53996
4,Receita % Velho,0.421443,0.416033,0.487883,0.332023,0.432079,0.400599,0.46004


In [264]:
receita_desvio = receita.copy()
receita_desvio.iloc[:, 1:] = (receita_desvio.iloc[:, 1:].div(receita_desvio['Média'], axis=0) - 1)
receita_desvio.drop('Média', axis=1, inplace=True)
receita_desvio.fillna(0, inplace=True)
receita_desvio

Data,Receita (USD),2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Receita Conteúdo novo (Soma),0.774491,0.037227,0.084358,-0.438735,-0.225682,-0.231659
1,Receita Conteúdo velho (Soma),0.727647,0.350398,-0.263421,-0.416442,-0.292782,-0.105401
2,Receita Total,0.754697,0.169557,-0.062595,-0.429315,-0.254035,-0.178309
3,Receita % Novo,0.009350,-0.114838,0.154557,-0.018384,0.036027,-0.066713
4,Receita % Velho,-0.012836,0.157650,-0.212177,0.025238,-0.049458,0.091583


In [265]:
receita.set_index('Receita (USD)', inplace=True)
receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
receita.loc['Receita % Velho'] = receita.loc['Receita % Velho'].map(format_as_percentage)
receita.reset_index(inplace=True)
receita = receita.astype(str)

#Formatação de porcentagens desvio
receita_desvio.iloc[:, 1:] = receita_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
receita_desvio_formatado = receita_desvio.iloc[:, 1:].map(format_as_percentage)
receita_desvio_formatado = pd.concat([receita_desvio['Receita (USD)'], receita_desvio_formatado], axis=1)
receita_desvio_formatado = receita_desvio_formatado.astype(str)

/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_46624/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '57.86%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_46624/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '58.40%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_46624/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '51.21%' has dtype incompat

# Tabela de Shorts

In [266]:
shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
shorts = shorts.drop(shorts.index[0])
shorts['Data'] = pd.to_datetime(shorts['Data']).dt.to_period('M')
shorts['Duração média da visualização'] = pd.to_timedelta(shorts['Duração média da visualização']).apply(lambda x: x.total_seconds())
shorts['Minutagem média'] = (shorts['Duração média da visualização'] * 100) / shorts['Porcentagem visualizada média (%)']
shorts['Taxa de cliques de impressões (%)'] = shorts['Taxa de cliques de impressões (%)'] / 100
shorts = shorts[['Data', 'Vídeos publicados', 'Impressões', 'Visualizações', 'Taxa de cliques de impressões (%)', 'RPM (USD)', 'Duração média da visualização', 'Minutagem média', 'Inscritos']]

shorts_publicados = shorts['Vídeos publicados'].mean()
shorts_impressões = shorts['Impressões'].mean()
shorts_views = shorts['Visualizações'].mean()
shorts_ctr = shorts['Taxa de cliques de impressões (%)'].mean()
shorts_rpm = shorts['RPM (USD)'].mean()
shorts_tempo_assistido = shorts['Duração média da visualização'].mean()
shorts_tempo_assistido = '{:02d}:{:02d}'.format(int(shorts_tempo_assistido // 60), int(shorts_tempo_assistido % 60))
shorts_minutagem = shorts['Minutagem média'].mean()
shorts_minutagem = '{:02d}:{:02d}'.format(int(shorts_minutagem // 60), int(shorts_minutagem % 60))
shorts_inscritos = shorts['Inscritos'].mean()

shorts['Duração média da visualização'] = shorts['Duração média da visualização'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
shorts['Duração média da visualização'] = pd.to_timedelta(shorts['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(shorts['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
shorts['Minutagem média'] = shorts['Minutagem média'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
shorts['Minutagem média'] = pd.to_timedelta(shorts['Minutagem média']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(shorts['Minutagem média']).dt.components['seconds'].astype(str).str.zfill(2)
shorts.sort_values(by='Data', inplace=True, ascending=True)
shorts.rename(columns = colunas_shorts, inplace = True)
# shorts['Tamanho de Vídeo Shorts'].loc[[6,5,4,3,2]]= '00:00'
# shorts['Tamanho de Vídeo Shorts'].loc[[1]]= '00:36'
shorts.set_index('Data', inplace = True)
shorts = shorts.T
shorts.insert(0, 'Média', [shorts_publicados, shorts_impressões, shorts_views, shorts_ctr, shorts_rpm, shorts_tempo_assistido, shorts_minutagem, shorts_inscritos])
shorts.reset_index(inplace=True)
shorts.rename(columns={'index':'Shorts'}, inplace=True)
shorts.fillna(0, inplace=True)
shorts

Data,Shorts,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de Shorts,20.333333,20,19,22,20,19,22
1,Impressões Shorts,17648078.0,23477468,17658424,16156081,13494653,10404422,24697420
2,Visualizações Shorts,11376333.5,16703175,13123955,9550334,8484613,4555549,15840375
3,Taxa de cliques de impressões (%),0.0476,0.0459,0.0466,0.0486,0.0495,0.0538,0.0412
4,RPM Shorts,0.032,0.031,0.032,0.028,0.031,0.034,0.036
5,Tempo Médio Assistido Shorts,00:50,00:51,00:50,00:50,00:50,00:49,00:51
6,Tamanho de Vídeo Shorts,00:58,00:57,00:57,00:58,00:59,00:57,00:58
7,Saldo de Inscritos Shorts,18017.0,32508,20175,14341,11129,6236,23713


In [267]:
shorts_temp = shorts.copy()
shorts_temp.set_index('Shorts', inplace=True)
shorts_temp.loc['Tamanho de Vídeo Shorts'] = shorts_temp.loc['Tamanho de Vídeo Shorts'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))
shorts_temp.loc['Tempo Médio Assistido Shorts'] = shorts_temp.loc['Tempo Médio Assistido Shorts'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))

shorts_desvio = shorts_temp.copy()
shorts_desvio.iloc[:, 1:] = (shorts_desvio.iloc[:, 1:].div(shorts_desvio['Média'], axis=0) - 1)

shorts_desvio.drop('Média', axis=1, inplace=True)
shorts_desvio.reset_index(drop=True, inplace=True)
shorts_desvio.insert(0, 'Shorts', shorts['Shorts'])
shorts_desvio.fillna(0, inplace=True)
shorts.fillna(0, inplace=True)
shorts_desvio

Data,Shorts,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de Shorts,-0.016393,-0.065574,0.081967,-0.016393,-0.065574,0.081967
1,Impressões Shorts,0.330313,0.000586,-0.084542,-0.235347,-0.41045,0.39944
2,Visualizações Shorts,0.468239,0.153619,-0.160509,-0.254187,-0.599559,0.392397
3,Taxa de cliques de impressões (%),-0.035714,-0.021008,0.021008,0.039916,0.130252,-0.134454
4,RPM Shorts,-0.03125,0.0,-0.125,-0.03125,0.0625,0.125
5,Tempo Médio Assistido Shorts,0.02,0.0,0.0,0.0,-0.02,0.02
6,Tamanho de Vídeo Shorts,-0.017241,-0.017241,0.0,0.017241,-0.017241,0.0
7,Saldo de Inscritos Shorts,0.804296,0.119776,-0.20403,-0.382306,-0.653882,0.316146


In [268]:
shorts.set_index('Shorts', inplace=True)
shorts.loc['Taxa de cliques de impressões (%)'] = shorts.loc['Taxa de cliques de impressões (%)'].map(format_as_percentage)
shorts.reset_index(inplace=True)
shorts = shorts.astype(str)

#Formatação de porcentagens desvio
shorts_desvio.iloc[:, 1:] = shorts_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
shorts_desvio_formatado = shorts_desvio.iloc[:, 1:].map(format_as_percentage)
shorts_desvio_formatado = pd.concat([shorts_desvio['Shorts'], shorts_desvio_formatado], axis=1)
shorts_desvio_formatado = shorts_desvio_formatado.astype(str)

# Tabela de Origem de Tráfego

In [269]:
origem = pd.read_csv(f'dados_full/{artista}/origem.csv')
colunas_ori = origem['Origem do tráfego'].unique().tolist()
colunas_origem = ['Data'] + colunas_ori

origem_pivot = origem.pivot_table(index='Data', columns='Origem do tráfego', values='Visualizações', aggfunc='first').reset_index()
origem_pivot.columns.name = None

colunas_desejadas = [
    'Recursos de navegação',
    'Vídeos sugeridos',
    'Páginas do canal',
    'Externa',
    'Notificações',
    'Pesquisa do YouTube',
    'Playlists',
    'Publicidade no YouTube'
]

# Usar copy() aqui para evitar SettingWithCopyWarning
origem_filtrado = origem_pivot[['Data'] + colunas_desejadas].copy()
origem_filtrado['Outros'] = origem_pivot.drop(['Data'] + colunas_desejadas, axis=1).sum(axis=1)
origem_filtrado = origem_filtrado.set_index('Data')
origem = origem_filtrado.copy()
origem

meses = origem_filtrado.index.tolist()
soma_origem = origem_filtrado.iloc[:, 0:].sum(axis=1)
soma_origem = pd.DataFrame(soma_origem)
soma_origem = soma_origem.rename(columns = {0:'Total'})
soma_origem.insert(0, 'Data', meses)
soma_origem.set_index('Data', inplace = True)
soma_origem

recursos_org = origem['Recursos de navegação'].mean()
sugeri_org = origem['Vídeos sugeridos'].mean()
pagina_org = origem['Páginas do canal'].mean()
extern_org = origem['Externa'].mean()
notifi_org = origem['Notificações'].mean()
pesquisa_org = origem['Pesquisa do YouTube'].mean()
playlist_org = origem['Playlists'].mean()
publi_org = origem['Publicidade no YouTube'].mean()
outros_org = origem['Outros'].mean()

origem.rename(columns={'Publicidade no YouTube':'Tráfego Pago'}, inplace=True)

origem = origem.T
origem.insert(0, 'Média', [recursos_org, sugeri_org, pagina_org, extern_org, notifi_org, pesquisa_org, playlist_org, publi_org,outros_org,])
origem.reset_index(inplace=True)
origem.rename(columns={'index':'Origem de Tráfego'}, inplace=True)
origem.fillna(0, inplace=True)
# origem['Média'] = origem['Média'].apply(lambda x: f'{x:.2f}')
origem

Data,Origem de Tráfego,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Recursos de navegação,1.697708e+06,2574701,2135856,1707678,945220,1344810,1477981
1,Vídeos sugeridos,7.784148e+05,1016922,870771,620944,509502,709648,942702
2,Páginas do canal,2.078902e+05,280575,218111,208474,190035,120224,229922
3,Externa,8.448700e+05,519432,634727,1426973,1533340,453503,501245
4,Notificações,7.703617e+04,96530,83747,102901,60484,62272,56283
5,Pesquisa do YouTube,2.581933e+05,331784,298520,264388,210704,173487,270277
6,Playlists,2.368088e+05,305631,324137,224657,151233,189518,225677
7,Tráfego Pago,0.000000e+00,0,0,0,0,0,0
8,Outros,2.099583e+05,211271,193331,186324,152237,121852,394735


In [270]:
origem_desvio = origem.copy()
origem_desvio.set_index('Origem de Tráfego', inplace=True)
origem_desvio.iloc[:, 1:] = (origem_desvio.iloc[:, 1:].div(origem_desvio['Média'], axis=0) - 1)
origem_desvio.drop('Média', axis=1, inplace=True)
origem_desvio.reset_index(drop=True, inplace=True)
origem_desvio.insert(0, 'Origem de Tráfego', origem['Origem de Tráfego'])
origem_desvio.fillna(0, inplace=True)
origem_desvio

Data,Origem de Tráfego,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Recursos de navegação,0.516575,0.258082,0.005873,-0.443237,-0.207867,-0.129426
1,Vídeos sugeridos,0.306401,0.118646,-0.202297,-0.345462,-0.088342,0.211053
2,Páginas do canal,0.349631,0.049165,0.002808,-0.085887,-0.421695,0.105978
3,Externa,-0.385193,-0.248728,0.688985,0.814883,-0.463227,-0.406719
4,Notificações,0.253048,0.087113,0.335749,-0.214862,-0.191652,-0.269395
5,Pesquisa do YouTube,0.285022,0.156188,0.023992,-0.183929,-0.328073,0.046801
6,Playlists,0.290623,0.368771,-0.051315,-0.361371,-0.199700,-0.047008
7,Tráfego Pago,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Outros,0.006252,-0.079193,-0.112567,-0.274918,-0.419637,0.880064


In [271]:
origem.iloc[:, 1:] = origem.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
origem['Média'] = origem['Média'].apply(lambda x: f'{x:.2f}')
perc_origem_formatado = origem.astype(str)

#Formatação de porcentagens desvio
origem_desvio.iloc[:, 1:] = origem_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
origem_desvio_formatado = origem_desvio.iloc[:, 1:].map(format_as_percentage)
origem_desvio_formatado = pd.concat([origem_desvio['Origem de Tráfego'], origem_desvio_formatado], axis=1)
origem_desvio_formatado = origem_desvio_formatado.astype(str)


# Comunidade

In [272]:
data_comu_2 = pd.read_csv('dados_full/'+artista+'/total.csv')
data_comu_2 = data_comu_2.drop(data_comu_2.index[0])
data_comu_2['Data'] = pd.to_datetime(data_comu_2['Data'])
data_comu_2['Mes'] = data_comu_2['Data'].dt.to_period('M')
data_comu_2.sort_values(by='Mes', inplace=True)
data_comu_2 = data_comu_2.drop('Data', axis=1)
colunas = data_comu_2.columns.tolist()
colunas = [colunas[-1]] + colunas[:-1]
data_comu_2 = data_comu_2[colunas]
data_comu_2


,Mes,"Média de ""Gostei"" da postagem (%)","""Gostei"" da postagem",Impressões da postagem,"Marcações ""Não gostei""",RPM (USD),Taxa de cliques de impressões (%),Impressões,Vídeos publicados,Inscritos,Porcentagem visualizada média (%),Visualizações,Tempo de exibição (horas),Duração média da visualização,Receita estimada (USD)
4,2023-11,0.36,18,5231,17899,0.440,7.83,60747820,39,52175,72.92,22040021,508665.3408,0:01:23,9699.639
5,2023-12,0.15,0,2700,13028,0.367,7.70,50865993,36,36630,71.65,17883155,411151.2832,0:01:22,6570.912
6,2024-01,0.15,-48,2041,10204,0.364,7.58,42668337,39,24830,62.62,14292673,345729.4874,0:01:27,5209.465
1,2024-02,3.39,5204,149321,9590,0.269,6.94,32345280,34,18760,63.09,12237368,248966.8555,0:01:13,3296.830
3,2024-03,0.79,4348,517742,5303,0.530,7.20,34694627,33,13656,67.15,7730863,233048.9174,0:01:48,4100.096
2,2024-04,0.81,9275,1068843,14275,0.246,6.41,53195920,36,40848,74.37,19939197,443816.5771,0:01:20,4900.259


In [273]:
data_comu = pd.read_csv('dados_full/'+artista+'/comunidade.csv')
data_comu.rename(columns=colunas_comunidade,inplace=True)
data_comu["Horário de publicação da postagem"] = pd.to_datetime(data_comu['Horário de publicação da postagem'], format="%b %d, %Y", errors='coerce')
data_comu.sort_values('Horário de publicação da postagem',ascending=False,inplace=True)
auxMeses_comu = mes.month+6
if auxMeses_comu > 12:
    auxMeses_comu = auxMeses_comu - 12 
comuCount = data_comu.loc[(data_comu["Horário de publicação da postagem"] > '2023-06')]
comuCount = comuCount.groupby([comuCount['Horário de publicação da postagem'].dt.month], sort=False).count()
df_graf_com = data_comu_2[['Mes','Impressões da postagem','"Gostei" da postagem','Média de "Gostei" da postagem (%)']].set_index('Mes').copy()
comuCount = comuCount[comuCount.index.isin(meses)]
auxComu = 1
auxList = []
for x in range(6):
    if len(comuCount.index) < auxComu:
        auxList.append(meses[x])
    else:
        if comuCount.index[-auxComu] == meses[x]:
            auxComu = auxComu + 1
        else:
            auxList.append(meses[x])
for x in auxList:
    comuCount.loc[x] = 0    
df_graf_com['Publicações na Comunidade'] = [comuCount['Postar'].loc[(comuCount.index == meses[-6])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-5])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-4])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-3])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-2])].item(),
                                           comuCount['Postar'].loc[(comuCount.index == meses[-1])].item()]
df_graf_com['Média de Impressões por postagem'] =  (df_graf_com['Impressões da postagem']/df_graf_com['Publicações na Comunidade'])
df_graf_com['Média de Impressões por postagem'] = df_graf_com['Média de Impressões por postagem'].replace([np.inf, -np.inf], 0)
df_graf_com = df_graf_com.fillna(0)
df_graf_com['Média de Impressões por postagem'] = df_graf_com['Média de Impressões por postagem'].astype(int)
df_graf_com = df_graf_com[['Publicações na Comunidade','Impressões da postagem','Média de Impressões por postagem','"Gostei" da postagem','Média de "Gostei" da postagem (%)']]

pubs_com = df_graf_com['Publicações na Comunidade'].mean()
impres_com = df_graf_com['Impressões da postagem'].mean()
impress_mean_com = df_graf_com['Média de Impressões por postagem'].mean()
likes_com = df_graf_com['"Gostei" da postagem'].mean()
likes_mean_com = df_graf_com['Média de "Gostei" da postagem (%)'].mean()


df_graf_com = df_graf_com.rename(columns={"Impressões da postagem": "Total de Impressões na Comunidade"})
df_graf_com = df_graf_com.T

df_graf_com.insert(0, 'Média', [pubs_com,impres_com,impress_mean_com,likes_com,likes_mean_com])
df_graf_com.reset_index(inplace=True)
df_graf_com.rename(columns={'index':'Comunidade'}, inplace=True)
df_graf_com

Mes,Comunidade,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Publicações na Comunidade,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
1,Total de Impressões na Comunidade,290979.666667,5231.00,2700.00,2041.00,149321.00,517742.00,1068843.00
2,Média de Impressões por postagem,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
3,"""Gostei"" da postagem",3132.833333,18.00,0.00,-48.00,5204.00,4348.00,9275.00
4,"Média de ""Gostei"" da postagem (%)",0.941667,0.36,0.15,0.15,3.39,0.79,0.81


In [274]:
comu_desvio = df_graf_com.copy()
comu_desvio.set_index('Comunidade', inplace=True)
comu_desvio.iloc[:, 1:] = (comu_desvio.iloc[:, 1:].div(comu_desvio['Média'], axis=0) - 1)
comu_desvio.drop('Média', axis=1, inplace=True)
comu_desvio.reset_index(drop=True, inplace=True)
comu_desvio.insert(0, 'Comunidade', df_graf_com['Comunidade'])
comu_desvio.fillna(0, inplace=True)
comu_desvio

Mes,Comunidade,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Publicações na Comunidade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Total de Impressões na Comunidade,-0.982023,-0.990721,-0.992986,-0.486834,0.779306,2.673257
2,Média de Impressões por postagem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"""Gostei"" da postagem",-0.994254,-1.000000,-1.015322,0.661116,0.387881,1.960579
4,"Média de ""Gostei"" da postagem (%)",-0.617699,-0.840708,-0.840708,2.600000,-0.161062,-0.139823


In [275]:
df_graf_com.iloc[:, 1:] = df_graf_com.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
df_graf_com['Média'] = df_graf_com['Média'].apply(lambda x: f'{x:.2f}')
df_graf_com_formatado = df_graf_com.astype(str)

#Formatação de porcentagens desvio
comu_desvio.iloc[:, 1:] = comu_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
comu_desvio_formatado = comu_desvio.iloc[:, 1:].map(format_as_percentage)
comu_desvio_formatado = pd.concat([comu_desvio['Comunidade'], comu_desvio_formatado], axis=1)
comu_desvio_formatado = comu_desvio_formatado.astype(str)

# Videos Publicados

In [276]:
vods = pd.read_csv(f'dados_full/{artista}/subs.csv')
vods = vods.drop(vods.index[0])
vods['Data'] = pd.to_datetime(vods['Data'])
vods['Data'] = vods['Data'].dt.to_period('M')
vods.sort_values(by='Data', inplace=True)
vods = vods[['Data', 'Vídeos publicados']]
vods['Vídeos publicados'] = vods['Vídeos publicados'].round(0)
vods.rename(columns = {'Vídeos publicados': "Número de VOD's"}, inplace=True)
vods.fillna(0,inplace=True)

p_shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
p_shorts = p_shorts.drop(p_shorts.index[0])
p_shorts['Data'] = pd.to_datetime(p_shorts['Data'])
p_shorts['Data'] = p_shorts['Data'].dt.to_period('M')
p_shorts.sort_values(by='Data', inplace=True)
p_shorts = p_shorts[['Data', 'Vídeos publicados']]
p_shorts.rename(columns = {'Vídeos publicados': 'Número de Shorts'}, inplace=True)
p_shorts.fillna(0, inplace=True)

p_lives = pd.read_csv(f'dados_full/{artista}/lives.csv')
p_lives = p_lives.drop(p_lives.index[0])
p_lives['Data'] = pd.to_datetime(p_lives['Data'])
p_lives['Data'] = p_lives['Data'].dt.to_period('M')
p_lives.sort_values(by='Data', inplace=True)
p_lives = p_lives[['Data', 'Vídeos publicados']]
p_lives.rename(columns = {'Vídeos publicados': 'Número de Lives'}, inplace=True)
p_lives.fillna(0, inplace=True)

publicados = pd.merge(vods, p_shorts[['Data', 'Número de Shorts']], on='Data', how='left')
publicados = pd.merge(publicados, p_lives[['Data', 'Número de Lives']], on='Data', how='left')
publicados.set_index('Data', inplace=True)

p_vod_mean = publicados["Número de VOD's"].mean()
p_shorts_mean = publicados["Número de Shorts"].mean()
p_lives_mean = publicados["Número de Lives"].mean()

publicados = publicados.T
publicados.insert(0,'Média', [p_vod_mean,p_shorts_mean,p_lives_mean])
publicados.reset_index(inplace=True)
publicados.rename(columns={'index':'Vídeos Publicados'}, inplace=True)
publicados.fillna(0, inplace = True)
publicados


Data,Vídeos Publicados,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de VOD's,15.833333,19.0,17.0,17.0,14.0,14.0,14.0
1,Número de Shorts,20.333333,20.0,19.0,22.0,20.0,19.0,22.0
2,Número de Lives,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [277]:
pubs_desvio = publicados.copy()
pubs_desvio.set_index('Vídeos Publicados', inplace=True)
pubs_desvio.iloc[:, 1:] = (pubs_desvio.iloc[:, 1:].div(pubs_desvio['Média'], axis=0) - 1)
pubs_desvio.drop('Média', axis=1, inplace=True)
pubs_desvio.reset_index(drop=True, inplace=True)
pubs_desvio.insert(0, 'Vídeos Publicados', publicados['Vídeos Publicados'])
pubs_desvio.fillna(0, inplace=True)
pubs_desvio

Data,Vídeos Publicados,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de VOD's,0.200000,0.073684,0.073684,-0.115789,-0.115789,-0.115789
1,Número de Shorts,-0.016393,-0.065574,0.081967,-0.016393,-0.065574,0.081967
2,Número de Lives,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [278]:
publicados.iloc[:, 1:] = publicados.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
publicados['Média'] = publicados['Média'].apply(lambda x: f'{x:.2f}')
publicados_formatado = publicados.astype(str)

#Formatação de porcentagens desvio
pubs_desvio.iloc[:, 1:] = pubs_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
pubs_desvio_formatado = pubs_desvio.iloc[:, 1:].map(format_as_percentage)
pubs_desvio_formatado = pd.concat([pubs_desvio['Vídeos Publicados'], pubs_desvio_formatado], axis=1)
pubs_desvio_formatado = pubs_desvio_formatado.astype(str)

# WatchTime

In [279]:
watch = pd.read_csv(f'dados_full/{artista}/total.csv')
watch = watch.drop(watch.index[0])
watch = watch[['Data', 'Tempo de exibição (horas)']].round(2)
watch.sort_values(by='Data', inplace=True)
watch.rename(columns={'Tempo de exibição (horas)':'WatchTime (horas)'}, inplace=True)
watch.set_index('Data', inplace=True)

watch_mean = watch['WatchTime (horas)'].mean()

watch = watch.T
watch.insert(0, 'Média', watch_mean)
watch.reset_index(inplace=True)
watch.rename(columns={'index':'WatchTime'}, inplace=True)
watch

Data,WatchTime,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,WatchTime (horas),365229.745,508665.34,411151.28,345729.49,248966.86,233048.92,443816.58


In [280]:
watch_desvio = watch.copy()
watch_desvio.set_index('WatchTime', inplace=True)
watch_desvio.iloc[:, 1:] = (watch_desvio.iloc[:, 1:].div(watch_desvio['Média'], axis=0) - 1)
watch_desvio.drop('Média', axis=1, inplace=True)
watch_desvio.reset_index(drop=True, inplace=True)
watch_desvio.insert(0, 'WatchTime', watch['WatchTime'])
watch_desvio.fillna(0, inplace=True)
watch_desvio

Data,WatchTime,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,WatchTime (horas),0.392727,0.125733,-0.053392,-0.318328,-0.361911,0.215171


In [281]:
watch.iloc[:, 1:] = watch.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
watch['Média'] = watch['Média'].apply(lambda x: f'{x:.2f}')
watch_formatado = watch.astype(str)

#Formatação de porcentagens desvio
watch_desvio.iloc[:, 1:] = watch_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
watch_desvio_formatado = watch_desvio.iloc[:, 1:].map(format_as_percentage)
watch_desvio_formatado = pd.concat([watch_desvio['WatchTime'], watch_desvio_formatado], axis=1)
watch_desvio_formatado = watch_desvio_formatado.astype(str)

# Inscritos

In [282]:
insc_vods = pd.read_csv(f'dados_full/{artista}/subs.csv')
insc_vods = insc_vods.drop(insc_vods.index[0])
insc_vods['Data'] = pd.to_datetime(insc_vods['Data'])
insc_vods['Data'] = insc_vods['Data'].dt.to_period('M')
insc_vods.sort_values(by='Data', inplace=True)
insc_vods = insc_vods[['Data', 'Inscritos']]
insc_vods['Inscritos'] = insc_vods['Inscritos'].round(0)
insc_vods.rename(columns = {'Inscritos': "Inscritos por VOD's"}, inplace=True)

insc_shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
insc_shorts = insc_shorts.drop(insc_shorts.index[0])
insc_shorts['Data'] = pd.to_datetime(insc_shorts['Data'])
insc_shorts['Data'] = insc_shorts['Data'].dt.to_period('M')
insc_shorts.sort_values(by='Data', inplace=True)
insc_shorts = insc_shorts[['Data', 'Inscritos']]
insc_shorts.rename(columns = {'Inscritos': 'Inscritos por Shorts'}, inplace=True)
insc_shorts.fillna(0, inplace=True)

insc_lives = pd.read_csv(f'dados_full/{artista}/lives.csv')
insc_lives = insc_lives.drop(insc_lives.index[0])
insc_lives['Data'] = pd.to_datetime(insc_lives['Data'])
insc_lives['Data'] = insc_lives['Data'].dt.to_period('M')
insc_lives.sort_values(by='Data', inplace=True)
insc_lives = insc_lives[['Data', 'Inscritos']]
insc_lives.rename(columns = {'Inscritos': 'Inscritos por Lives'}, inplace=True)
insc_lives.fillna(0, inplace=True)

inscritos = pd.merge(insc_vods, insc_shorts[['Data', 'Inscritos por Shorts']], on='Data', how='left')
inscritos = pd.merge(inscritos, insc_lives[['Data', 'Inscritos por Lives']], on='Data', how='left')
inscritos.set_index('Data', inplace=True)

insc_vods_mean = inscritos["Inscritos por VOD's"].mean()
insc_shorts_mean = inscritos["Inscritos por Shorts"].mean()
insc_lives_mean = inscritos["Inscritos por Lives"].mean()

inscritos = inscritos.T
inscritos.insert(0,'Média',[insc_vods_mean, insc_shorts_mean, insc_lives_mean])
inscritos.reset_index(inplace=True)
inscritos.rename(columns={'index':'Inscritos'}, inplace=True)
inscritos.fillna(0, inplace=True)
inscritos

Data,Inscritos,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Inscritos por VOD's,11467.166667,16402.0,13856.0,9701.0,7259.0,7463.0,14122.0
1,Inscritos por Shorts,18017.000000,32508.0,20175.0,14341.0,11129.0,6236.0,23713.0
2,Inscritos por Lives,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [283]:
insc_desvio = inscritos.copy()
insc_desvio.set_index('Inscritos', inplace=True)
insc_desvio.iloc[:, 1:] = (insc_desvio.iloc[:, 1:].div(insc_desvio['Média'], axis=0) - 1)
insc_desvio.drop('Média', axis=1, inplace=True)
insc_desvio.reset_index(drop=True, inplace=True)
insc_desvio.insert(0, 'Inscritos', inscritos['Inscritos'])
insc_desvio.fillna(0, inplace=True)
insc_desvio

Data,Inscritos,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Inscritos por VOD's,0.430345,0.208319,-0.154019,-0.366975,-0.349185,0.231516
1,Inscritos por Shorts,0.804296,0.119776,-0.204030,-0.382306,-0.653882,0.316146
2,Inscritos por Lives,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [284]:
inscritos.iloc[:, 1:] = inscritos.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
inscritos['Média'] = inscritos['Média'].apply(lambda x: f'{x:.2f}')
inscritos_formatado = inscritos.astype(str)

#Formatação de porcentagens desvio
insc_desvio.iloc[:, 1:] = insc_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
insc_desvio_formatado = insc_desvio.iloc[:, 1:].map(format_as_percentage)
insc_desvio_formatado = pd.concat([insc_desvio['Inscritos'], insc_desvio_formatado], axis=1)
insc_desvio_formatado = insc_desvio_formatado.astype(str)

# Montagem do arquivo

In [285]:
with pd.ExcelWriter(f'exports_tabelas/Monthly Abril 2024 ({artista}).xlsx') as writer:
    
    data.to_excel(writer, sheet_name='Resultado', index=False)

    num_rows_df1 = data.shape[0]

    conteudo.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1+1, index=False)

    num_rows_df2 = conteudo.shape[0]

    receita.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + 2, index=False)

    num_rows_df3 = receita.shape[0]

    shorts.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + 3, index=False)

    num_rows_df4 = shorts.shape[0]

    perc_origem_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + 4, index=False)

    num_rows_df5 = perc_origem_formatado.shape[0]
    
    df_graf_com_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + 5, index=False)

    num_rows_df6 = df_graf_com_formatado.shape[0]

    publicados_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + 6, index=False)

    num_rows_df7 = publicados_formatado.shape[0]

    watch_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + num_rows_df7 + 7, index=False)

    num_rows_df8 = watch_formatado.shape[0]

    inscritos_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + num_rows_df7 + num_rows_df8 + 8, index=False)
    
    
    data_desvio_formatado.to_excel(writer, sheet_name='Desvio', index = False)

    num_rows_desv_df1 = data_desvio_formatado.shape[0]

    conteudo_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_desv_df1 + 1, index=False)

    num_rows_desv_df2 = conteudo_desvio_formatado.shape[0]

    receita_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_desv_df1 + num_rows_desv_df2 + 2, index=False)

    num_rows_desv_df3 = receita_desvio_formatado.shape[0]

    shorts_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + 3, index=False)

    num_rows_desv_df4 = shorts_desvio_formatado.shape[0]

    origem_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + 4, index=False)

    num_rows_desv_df5 = origem_desvio_formatado.shape[0]

    comu_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + 5, index=False)

    num_rows_desv_df6 = comu_desvio_formatado.shape[0]
    
    pubs_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + 6, index=False)

    num_rows_desv_df7 = pubs_desvio_formatado.shape[0]

    watch_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + num_rows_desv_df7 + 7, index=False)

    num_rows_desv_df8 = watch_desvio_formatado.shape[0]

    insc_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + num_rows_desv_df7 + num_rows_desv_df8 + 8, index=False)



# TABELA PARA QUADRANTE

In [286]:
quadrante = pd.read_csv(f'dados_full/{artista}/total.csv')
quadrante = quadrante.drop(quadrante.index[0])
quadrante.set_index('Data', inplace=True)
quadrante = quadrante[['Inscritos','RPM (USD)','Visualizações','Receita estimada (USD)']].round(2)


insc_q_mean = quadrante['Inscritos'].mean()
rpm_q_mean = quadrante['RPM (USD)'].mean()
views_q_mean = quadrante['Visualizações'].mean()
reven_q_mean = quadrante['Receita estimada (USD)'].mean()

quadrante = quadrante.T
quadrante.insert(0, 'Média', [insc_q_mean, rpm_q_mean, views_q_mean, reven_q_mean])
quadrante.fillna(0,inplace=True)
quadrante.reset_index(inplace=True)

quadrante_desvio = quadrante.copy()
quadrante_desvio.iloc[:, 1:] = (quadrante_desvio.iloc[:, 1:].div(quadrante_desvio['Média'], axis=0) - 1)
quadrante_desvio.drop('Média', axis=1, inplace=True)
quadrante_desvio.reset_index(drop=True, inplace=True)
quadrante_desvio.insert(0, 'Data', quadrante['index'])
quadrante_desvio.fillna(0, inplace=True)
quadrante_desvio.iloc[:, 1:] = quadrante_desvio.iloc[:, 1:].replace('.', ',', regex=True)
quadrante_desvio.drop('index', axis=1, inplace=True)
quadrante_desvio

Data,Data,2024-02,2024-04,2024-03,2023-11,2023-12,2024-01
0,Inscritos,-0.397750,0.311339,-0.561603,0.674969,1.759292e-01,-0.202885
1,RPM (USD),-0.270270,-0.324324,0.432432,0.189189,2.220446e-16,-0.027027
2,Visualizações,-0.219914,0.271048,-0.507187,0.404967,1.399829e-01,-0.088897
3,Receita estimada (USD),-0.414369,-0.129544,-0.271680,0.722992,1.672211e-01,-0.074620


In [287]:
with pd.ExcelWriter(f'dados_full/{artista}/quadrante.xlsx') as writer:
    
    quadrante.to_excel(writer, sheet_name='Resultado', index=False)

    quadrante_desvio.to_excel(writer, sheet_name='Desvio', index=False)